In [1]:
# Built-in libraries
from datetime import datetime, timedelta

# NumPy, SciPy and Pandas
import pandas as pd
import numpy as np


In [174]:
def hourly_dataset(name):
    """
    Constants for time period with maximum number of buildings measured simultaneously in the BDG dataset.
    For more details, go to old_files/RawFeatures_BDG.ipynb
    """
    BDG_STARTDATE = datetime.strptime('01/01/15 00:00', '%d/%m/%y %H:%M')
    BDG_ENDDATE = datetime.strptime('30/11/15 23:00','%d/%m/%y %H:%M')

    # Building Data Genome dataset
    if name == 'BDG':
        df = pd.read_csv('../data/raw/temp_open_utc_complete.csv', parse_dates=True, 
                         infer_datetime_format=True, index_col=0)
        # truncate the dataframe based on a pre-calculated time period, if needed
        startDate = BDG_STARTDATE
        endDate = BDG_ENDDATE
        df = df[(df.index >= startDate) & (df.index <= endDate)]
    
    # Washington D.C. dataset
    elif name == 'DGS':
        df = pd.read_csv('../data/raw/DGS_322_Buildings-15m-By_Building-DST-gap-filled-3-2-18-508pm.csv',
                            parse_dates=[['Building ID', 'Unnamed: 1']], infer_datetime_format=True)
        # get rid of temperature column
        del df['Unnamed: 2']

        # update column names to match the row of building names
        new_column_names = df.iloc[0,:]
        df.columns = new_column_names

        # get rid of rows with metadata and update index
        df = df.drop([0,1,2], axis=0)
        df = df.rename(columns = {'Building nan':'timestamp'})
        df.index = df['timestamp'].astype('datetime64[ns]')
        del df['timestamp']
        df = df.astype(float)
        
        # since the dataset is made from 15min interval readings, resample to 1 hr
        df = df.resample('1H').sum()
             
    else:
        print("Please choose a valid dataset")
        exit()
    
    # save the file to csv before exit
    df.to_csv('../data/processed/{}_dataset.csv'.format(name))
    
    return df


In [12]:
from collections import Counter


def resampleDGS():
    df = pd.read_csv("../data/processed/DGS_dataset.csv", parse_dates=True, infer_datetime_format=True, index_col=0)
    og_index = df.index.values

    df = df.T
    df_meta = pd.read_csv('../data/raw/dgs_metadata.csv')
        
    df_aux = pd.read_csv("../data/raw/DGS_322_Buildings-15m-By_Building-DST-gap-filled-3-2-18-508pm.csv")

    # get labels for all buildings
    df_aux = df_aux.T
    df_aux_og = df_aux.copy()
    df_label = df_aux[df_aux.iloc[:, 0].isin(df.index.values)] # get id based on names
    df_label = df_meta[df_meta['id'].isin(df_label.index.values)] # get label based on id    
    
#     print(c.value_counts())
    
    cnt = Counter(df_label['espm_type_name'
                          ])
    for i in df_label['espm_type_name']:
        print(cnt[i])
        
    df_label = df_label[(df_label['espm_type_name'] == 'K-12 School') |
                (df_label['espm_type_name'] == 'Other - Recreation') |
                (df_label['espm_type_name'] == 'Fire Station') |
                (df_label['espm_type_name'] == 'Office') |
                (df_label['espm_type_name'] == 'Library') |
                (df_label['espm_type_name'] == 'Other - Public Services') |
                (df_label['espm_type_name'] == 'Police Station')]
    
#     print(df_label['espm_type_name'].value_counts())
          
    df_aux_og = df_aux_og.drop(df_aux_og.index[0:3])
    df_aux_og.index = list(map(int, df_aux_og.index.values))
    df_bdg_name = df_aux_og[df_aux_og.index.isin(df_label['id'])]
    
    df = df[df.index.isin(df_bdg_name.iloc[:, 0])]
    df = df.T
    df.index = og_index
#     df.to_csv('../data/processed/DGS_dataset.csv')

                      

In [188]:
# load building gnome dataset (BDG)
df_BDG = hourly_dataset('BDG')

# load dc building dataset (DC)
df_DGS = hourly_dataset('DGS')


/Users/matias/miniconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3185: DtypeWarning: Columns (2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249

In [13]:
resampleDGS()

13
20
46
26
117
117
46
117
26
20
117
117
117
46
117
117
13
117
46
46
26
117
32
117
32
46
20
117
46
117
117
46
20
26
32
117
32
117
117
117
46
46
117
117
46
117
117
117
117
117
117
26
117
46
117
46
117
20
117
117
46
46
117
117
32
13
117
117
117
117
117
117
117
20
46
117
32
117
117
32
20
46
117
117
20
26
26
32
32
46
117
46
117
117
117
26
32
46
117
20
20
117
26
117
117
117
46
32
117
117
32
46
32
117
32
117
117
117
46
117
117
117
46
117
46
46
117
117
32
117
46
117
46
20
46
117
46
46
117
32
32
13
46
117
26
46
32
26
20
20
117
117
32
117
13
20
26
117
46
117
32
20
13
117
46
46
20
117
117
13
20
117
32
32
46
13
20
13
26
32
20
20
117
117
117
13
32
26
46
32
32
13
26
117
117
117
20
20
20
117
117
117
117
117
117
117
117
117
117
117
117
117
20
32
46
117
46
117
117
117
117
20
13
20
117
20
26
26
20
26
46
20
32
20
32
32
32
13
32
20
117
20
117
46
46
117
46
117
117
46
20
117
20
20
20
26
26
20
117
117
26
20
46
117
26
117
20
26
117
26
26
20
117
26
